# Creating and Managing Messages
An Assistant represents an entity that can be configured to respond to a user's messages using several parameters like model, instructions, and tools. 

A Thread represents a conversation between a user and one or many Assistants. You can create a Thread when a user (or your AI application) starts a conversation with your Assistant.

The contents of the messages your users or applications create are added as Message objects to the Thread. Messages can contain both text and files. There is no limit to the number of Messages you can add to Threads — we smartly truncate any context that does not fit into the model's context window.

## Creating an Assistant and a Thread Review
Let's create an Assistant and Thread to get us set up for success with our Messages. 

We begin with an Assistant. 

In [4]:
from openai import OpenAI

# Create an instance of the OpenAI class
# This assumes you have the OPENAI_API_KEY environment variable set
client = OpenAI()

# Create an assistant that uses code interpreter.
assistant = client.beta.assistants.create(
    model="gpt-4-turbo",
    instructions="You are a helpful assistant.",
    name="Message Holder",
    metadata={
        "holds_messages": "True",
        "likes_messages": "True",
    },
    temperature=1,
    top_p=1,
)

# Print the details of the created assistant to check the properties.
print(assistant)
print("\n\n")
print(assistant.name)
print(assistant.metadata)

Assistant(id='asst_sfNDj5MsHypnXwrpQkV2gR2F', created_at=1715466848, description=None, instructions='You are a helpful assistant.', metadata={'holds_messages': 'True', 'likes_messages': 'True'}, model='gpt-4-turbo', name='Message Holder', object='assistant', tools=[], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0)



Message Holder
{'holds_messages': 'True', 'likes_messages': 'True'}


Now we create an empty thread that will hold our messages. Remember to hang on to that thread ID.

In [13]:
from openai import OpenAI

# Initialize a client instance of the OpenAI API
client = OpenAI()

# Create a thread using the OpenAI API and store it in a variable
# The metadata specifies a user identifier
thread_holding_messages = client.beta.threads.create(
    metadata={
        "user": "abc123"
    }
)

# Output the result of the thread creation to the console
print(thread_holding_messages)


Thread(id='thread_Sm0jLjJVquSIk4kdMg2hdJPl', created_at=1715467151, metadata={'user': 'abc123'}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


## Creating Messages
To create a message we need to determine which thread to add the message to, the role for the message, and the actual message itself. 

Here is a simple message with some metadata.

In [24]:
message = client.beta.threads.messages.create(
  thread_id=thread_holding_messages.id,
  role="user",
  content="What is a penguin?",
  metadata={"key": "value"}
)

print(message)
print("\n")
print(message.id)
print(message.content)
print(message.content[0].text.value)
print(message.role)

Message(id='msg_mWtF7z47eNkOVoA5j4RkAsL4', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What is a penguin?'), type='text')], created_at=1715467481, incomplete_at=None, incomplete_details=None, metadata={'key': 'value'}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_Sm0jLjJVquSIk4kdMg2hdJPl')


msg_mWtF7z47eNkOVoA5j4RkAsL4
[TextContentBlock(text=Text(annotations=[], value='What is a penguin?'), type='text')]
What is a penguin?
user


Now let's create a more complex message that uses Vision to get an answer.

In [30]:
file = client.files.create(
    file=open("./artifacts/PuppyDog.jpg", "rb"),
    purpose="vision"
)

message_vision = client.beta.threads.messages.create(
    thread_id=thread_holding_messages.id,
    role="user",
    content=[
        {
            "type": "text",
            "text": "What is the difference between these images?"
        },
        {
            "type": "image_url",
            "image_url": {"url": "https://en.wikipedia.org/wiki/File:Cat_August_2010-4.jpg"}
        },
        {
            "type": "image_file",
            "image_file": {"file_id": file.id}  
        },
    ],
)
